In [4]:
# =========================================
# Colab: Local Hugging Face LLM
# Uniform and Packaging Assessment (Fixed JSON)
# =========================================

# Install dependencies
!pip install --quiet transformers accelerate sentencepiece python-docx matplotlib json5

import os
import json
import json5 # Import json5
import re
from pathlib import Path
from transformers import pipeline
from docx import Document
from docx.shared import Inches
import matplotlib.pyplot as plt
from google.colab import files
import textwrap

# ---------- Output directories ----------
OUT_DIR = Path("output")
IMG_DIR = OUT_DIR / "images"
OUT_DIR.mkdir(exist_ok=True)
IMG_DIR.mkdir(exist_ok=True)

# ---------- Load local model ----------
print("Loading Hugging Face model (this may take a minute)...")
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    device_map="auto"
)
print("Model loaded ✅")

# ---------- Function to draw packed spheres ----------
def draw_packed_spheres(rows, cols, radius_cm, padding_cm=0.5, filename="packed_spheres.png"):
    scale = 30  # px per cm
    r_px = radius_cm * scale
    pad_px = padding_cm * scale
    width = (cols * 2 * r_px) + (2 * pad_px)
    height = (rows * 2 * r_px) + (2 * pad_px)

    fig, ax = plt.subplots(figsize=(width/100, height/100), dpi=100)
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.set_aspect('equal')
    ax.axis('off')

    for i in range(rows):
        for j in range(cols):
            center_x = pad_px + r_px + j*(2*r_px)
            center_y = pad_px + r_px + i*(2*r_px)
            ax.add_patch(plt.Circle((center_x, center_y), r_px, fill=False, linewidth=2))

    ax.add_patch(plt.Rectangle((0, 0), width, height, fill=False, linewidth=2, linestyle='--'))
    path = IMG_DIR / filename
    plt.savefig(path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    return path

# ---------- Prompt for the model ----------
prompt = textwrap.dedent("""
Return ONLY valid JSON. No extra words. Start with { and end with }.
The JSON must match this structure exactly:

{
   "title":"...",
   "instructions":"...",
   "difficulty":"...",
   "options":[
      "...",
      "...",
      "...",
      "..."
   ],
   "correct_option":"...",
   "explanation":"...",
   "table":[
      [
         "...",
         "..."
      ],
      [
         "...",
         "..."
      ],
      [
         "...",
         "..."
      ],
      [
         "...",
         "..."
      ]
   ],
   "image":{
      "type":"packed_spheres",
      "rows":2,
      "cols":4,
      "radius_cm":3,
      "filename":"uniform_pack.png"
   }
}

Fill it with a Uniform and Packaging math question about shirt and pants combinations and a packaging problem with spheres.
""")

# ---------- Generate text from model ----------
print("Generating question...")
out = generator(prompt, max_new_tokens=512, do_sample=False, temperature=0.0)
raw_text = out[0]["generated_text"].strip()
print("Raw model output:\n", raw_text[:500])

# ---------- JSON Repair Step ----------
json_text = raw_text.strip()
json_text = re.sub(r"^```(?:json)?\s*", "", json_text)
json_text = re.sub(r"\s*```$", "", json_text)

# Ensure keys are quoted and handle missing colons before values
json_text = re.sub(r'(\w+)\s*:\s*', r'"\1": ', json_text)
json_text = re.sub(r'(\w+)\s*\[', r'"\1": [', json_text) # Handle cases like "table"[...

# Ensure it starts and ends with braces
if not json_text.startswith("{"):
    json_text = "{" + json_text
if not json_text.endswith("}"):
    json_text = json_text + "}"


# Parse JSON using json5
try:
    data = json5.loads(json_text)
except Exception as e: # Catch broader exception for easier debugging
    raise ValueError(f"Could not parse model output as JSON.\nAttempted JSON:\n{json_text}") from e

# ---------- Create Word document ----------
doc = Document()
doc.add_heading(data.get("title", "Uniform and Packaging Math Assessment"), level=1)
doc.add_paragraph(data.get("instructions", ""))
doc.add_paragraph("Difficulty: " + data.get("difficulty", ""))
doc.add_paragraph("Options:")
for opt in data.get("options", []):
    doc.add_paragraph(opt)
doc.add_paragraph("Correct Answer: " + data.get("correct_option", ""))
doc.add_paragraph("Explanation: " + data.get("explanation", ""))

# Add table if present
if "table" in data:
    table = doc.add_table(rows=1, cols=2)
    table.style = 'Table Grid'
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Shirt Color'
    hdr_cells[1].text = 'Pants Color'
    # Ensure data["table"] is a list of lists
    if isinstance(data["table"], list) and all(isinstance(row, list) for row in data["table"]):
        for row in data["table"]:
            # Ensure rows have at least two elements
            if len(row) >= 2:
                r = table.add_row().cells
                r[0].text = str(row[0]) # Convert to string to handle potential non-string data
                r[1].text = str(row[1]) # Convert to string
            else:
                print(f"Skipping malformed table row: {row}")
    else:
        print("Warning: 'table' data is not in the expected list of lists format.")


# Add image if present
img_info = data.get("image")
if img_info and img_info.get("type") == "packed_spheres":
    img_path = draw_packed_spheres(
        img_info.get("rows", 2),
        img_info.get("cols", 4),
        img_info.get("radius_cm", 3),
        filename=img_info.get("filename", "pack.png")
    )
    doc.add_picture(str(img_path), width=Inches(3.0))
    doc.add_paragraph(f"Associated diagram: {img_info.get('filename')}")

# ---------- Save and download ----------
doc_path = OUT_DIR / "Uniform_and_Packaging_LLM.docx"
doc.save(doc_path)
print(f"Word document saved as {doc_path}")

files.download(str(doc_path))
if img_info:
    files.download(str(img_path))

Loading Hugging Face model (this may take a minute)...


Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model loaded ✅
Generating question...
Raw model output:
 "title":"...", "instructions":"...", "difficulty":"...", "options":["...", "...", "...", "..."], "correct_option":"...", "explanation":"...", "table":["...", "..."], ["...", "..."], ["...", "..."], ["...", "..."]], "image": "type":"packed_spheres", "rows":2, "cols":4, "radius_cm":3, "filename":"uniform_pack.png"


ValueError: Could not parse model output as JSON.
Attempted JSON:
{"title":"...", "instructions":"...", "difficulty":"...", "options":["...", "...", "...", "..."], "correct_option":"...", "explanation":"...", "table":["...", "..."], ["...", "..."], ["...", "..."], ["...", "..."]], "image": "type":"packed_spheres", "rows":2, "cols":4, "radius_cm":3, "filename":"uniform_pack.png"}

In [6]:
from docx import Document
from docx.shared import Inches

# Create a new Word document
doc = Document()

# First question block
doc.add_heading("Uniform Combinations at Greenfield Academy", level=1)
doc.add_paragraph("Determine the number of unique uniform combinations based on shirt and pant color options.")
doc.add_paragraph("Each student at Greenfield Academy wears a uniform consisting of 1 shirt and 1 pair of pants. The table below shows the available colors for each item. How many different uniform combinations are possible?")
doc.add_paragraph("Instruction: Choose the correct number of uniform combinations.")
doc.add_paragraph("Difficulty: Easy")
doc.add_paragraph("Options:")
doc.add_paragraph("1. Eight")
doc.add_paragraph("2. Ten")
doc.add_paragraph("3. Sixteen (Correct)")
doc.add_paragraph("4. Twelve")
doc.add_paragraph("Explanation: There are 4 shirt colors and 4 pant colors. The total combinations are \(4 × 4 = 16\).")
doc.add_paragraph("Subject: Quantitative Math")
doc.add_paragraph("Unit: Problem Solving")
doc.add_paragraph("Topic: Numbers and Operations")
doc.add_paragraph("Marks: 1")

# Add a table for uniform options
table = doc.add_table(rows=1, cols=2)
table.style = 'Table Grid'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Shirt Color'
hdr_cells[1].text = 'Pants Color'
uniform_data = [("Blue", "Gray"), ("Green", "Black"), ("White", "Navy"), ("Orange", "Khaki")]
for shirt, pants in uniform_data:
    row_cells = table.add_row().cells
    row_cells[0].text = shirt
    row_cells[1].text = pants

doc.add_paragraph("\n")

# Second question block
doc.add_heading("Dimensions of a Box of Spheres", level=1)
doc.add_paragraph("Estimate the dimensions of a rectangular box containing tightly packed spheres.")
doc.add_paragraph("The top view of a rectangular box containing 8 tightly packed spheres is shown. Each sphere has a radius of 1.5 cm. What are the closest dimensions, in centimeters, of the rectangular box?")
doc.add_paragraph("Instruction: Choose the closest dimensions of the box.")
doc.add_paragraph("Difficulty: Moderate")
doc.add_paragraph("Options:")
doc.add_paragraph("1. \(3 × 6 × 6\)")
doc.add_paragraph("2. \(3 × 12 × 6\)")
doc.add_paragraph("3. \(3 × 6 × 12\) (Correct)")
doc.add_paragraph("4. \(6 × 6 × 12\)")
doc.add_paragraph("Explanation: Each sphere has a diameter of \(2 × 1.5 = 3\) cm. If 4 spheres are packed along the length and 2 along the width, the dimensions are approximately \(3 × 6 × 12\) cm.")
doc.add_paragraph("Subject: Quantitative Math")
doc.add_paragraph("Unit: Geometry and Measurement")
doc.add_paragraph("Topic: Solid Figures (Volume of Cubes)")
doc.add_paragraph("Marks: 1")

# Save the document
output_path = "math_assessment_questions.docx"
doc.save(output_path)

print(f"Word document saved as {output_path}")



Word document saved as math_assessment_questions.docx


In [5]:
# =========================================
# Colab: Local Hugging Face LLM
# Uniform and Packaging Assessment (No JSON)
# =========================================

# Install dependencies
!pip install --quiet transformers accelerate sentencepiece python-docx matplotlib

import os
from pathlib import Path
from transformers import pipeline
from docx import Document
from docx.shared import Inches
import matplotlib.pyplot as plt
from google.colab import files
import textwrap

# ---------- Output directories ----------
OUT_DIR = Path("output")
IMG_DIR = OUT_DIR / "images"
OUT_DIR.mkdir(exist_ok=True)
IMG_DIR.mkdir(exist_ok=True)

# ---------- Load local model ----------
print("Loading Hugging Face model (this may take a minute)...")
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    device_map="auto"
)
print("Model loaded ✅")

# ---------- Function to draw packed spheres ----------
def draw_packed_spheres(rows, cols, radius_cm, padding_cm=0.5, filename="packed_spheres.png"):
    scale = 30  # px per cm
    r_px = radius_cm * scale
    pad_px = padding_cm * scale
    width = (cols * 2 * r_px) + (2 * pad_px)
    height = (rows * 2 * r_px) + (2 * pad_px)

    fig, ax = plt.subplots(figsize=(width/100, height/100), dpi=100)
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.set_aspect('equal')
    ax.axis('off')

    for i in range(rows):
        for j in range(cols):
            center_x = pad_px + r_px + j*(2*r_px)
            center_y = pad_px + r_px + i*(2*r_px)
            ax.add_patch(plt.Circle((center_x, center_y), r_px, fill=False, linewidth=2))

    ax.add_patch(plt.Rectangle((0, 0), width, height, fill=False, linewidth=2, linestyle='--'))
    path = IMG_DIR / filename
    plt.savefig(path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    return path

# ---------- Prompt for plain text output ----------
prompt = textwrap.dedent("""
Write a math assessment titled "Uniform and Packaging Math Assessment".
Include exactly two questions:

Question 1: A uniform combination counting problem about shirts and pants.
- Include a small table of shirt and pant colors.
- Provide 4 multiple choice options (A-D).
- Include the correct answer and explanation.

Question 2: A packaging problem with 8 tightly packed spheres (2 rows × 4 columns) of radius 3 cm.
- Provide 4 multiple choice options (A-D).
- Include the correct answer and explanation.
- Say that a diagram will be attached.

Format the output clearly in text so it can be inserted directly into a Word document.
""")

print("Generating questions...")
out = generator(prompt, max_new_tokens=600, do_sample=False, temperature=0.0)
generated_text = out[0]["generated_text"].strip()
print("Generated text:\n", generated_text)

# ---------- Create Word document ----------
doc = Document()
doc.add_heading("Uniform and Packaging Math Assessment", level=1)

# Add the generated text
for line in generated_text.split("\n"):
    if line.strip():
        doc.add_paragraph(line.strip())

# Add the packed spheres diagram for Question 2
image_path = draw_packed_spheres(rows=2, cols=4, radius_cm=3)
doc.add_picture(str(image_path), width=Inches(3.0))
doc.add_paragraph("Diagram: Top view of 8 tightly packed spheres.")

# Save and download
doc_path = OUT_DIR / "Uniform_and_Packaging_Assessment.docx"
doc.save(doc_path)
print(f"Word document saved as {doc_path}")

files.download(str(doc_path))
files.download(str(image_path))

Loading Hugging Face model (this may take a minute)...


Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model loaded ✅
Generating questions...
Generated text:
 Include the correct answer and explanation.
Word document saved as output/Uniform_and_Packaging_Assessment.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>